# 平均報酬強化学習

参考：
* [Average-reward model-free reinforcement learning: a systematic review and literature mapping](https://arxiv.org/abs/2010.08920)

今回は平均報酬強化学習について学びます．大体の証明はPuterman 1994にあるっぽい？

## 定常分布について

次を定義します：
* 方策から誘導されるマルコフ連鎖：$\boldsymbol{P}_\pi \in[0,1]^{|\mathcal{S}| \times|\mathcal{S}|}$
    * その$t$回繰り返したもの：$\boldsymbol{P}_\pi^t\left[s_0, s\right]=\operatorname{Pr}\left\{S_t=s \mid s_0, \pi\right\}=: p_\pi^t\left(s \mid s_0\right)$

このとき，このマルコフ連鎖の収束先は
$$p_\pi^{\star}\left(s \mid s_0\right)=\lim _{t_{\max } \rightarrow \infty} \frac{1}{t_{\max }} \sum_{t=0}^{t_{\max }-1} p_\pi^t\left(s \mid s_0\right)=\lim _{t_{\max } \rightarrow \infty} p_\pi^{t_{\max }}\left(s \mid s_0\right), \quad \forall s \in \mathcal{S}$$

を満たします．この最初のlimitは有限なMDPならば必ず存在し，２つ目のLimitはMDPがaperiodicであれば存在します（TODO: 証明）

また，$p_\pi^\star(s|s_0)$をベクトル化したものを$\boldsymbol{p}_\pi^{\star} \in[0,1]^{|\mathcal{S}|}$とします．
これは
$$\left(\boldsymbol{p}_\pi^{\star}\right)^{\top} \boldsymbol{P}_\pi=\left(\boldsymbol{p}_\pi^{\star}\right)^{\top}$$
を満たします

## ゲインについて

平均報酬はゲインとも呼ばれ，次で定義され，以下の変形が成立します：

$$
\begin{aligned}
v_g\left(\pi, s_0\right) & :=\lim _{t_{\max } \rightarrow \infty} \frac{1}{t_{\max }} \mathbb{E}_{S_t, A_t}\left[\sum_{t=0}^{t_{\max }-1} r\left(S_t, A_t\right) \mid S_0=s_0, \pi\right] \\
& =\sum_{s \in \mathcal{S}}\left\{\lim _{t_{\max } \rightarrow \infty} \frac{1}{t_{\max }} \sum_{t=0}^{t_{\max }-1} p_\pi^t\left(s \mid s_0\right)\right\} r_\pi(s)=\sum_{s \in \mathcal{S}} p_\pi^{\star}\left(s \mid s_0\right) r_\pi(s) \\
& =\lim _{t_{\max } \rightarrow \infty} \mathbb{E}_{S_{t_{\max }}, A_{t_{\max }}}\left[r\left(S_{t_{\max }}, A_{t_{\max }}\right) \mid S_0=s_0, \pi\right]
\end{aligned}
$$

ここで，
* $r_\pi(s)=\sum_{a \in \mathcal{A}} \pi(a \mid s) r(s, a)$
* 最初のlimitは方策が定常かつMDPが有限であれば存在します（TODO: 証明）
* ２つ目のLimitは簡単です
* ３つ目は方策が定常かつMDPがaperiodic，もしくは報酬に何らかの構造が入っている場合に存在します（TODO: 証明）

行列形式で書けば，

$$
\boldsymbol{v}_g(\pi)=\lim _{t_{\max } \rightarrow \infty} \frac{1}{t_{\max }} \boldsymbol{v}_{t_{\max }}(\pi)=\lim _{t_{\max } \rightarrow \infty} \frac{1}{t_{\max }} \sum_{t=0}^{t_{\max }-1} \boldsymbol{P}_\pi^t \boldsymbol{r}_\pi=\boldsymbol{P}_\pi^{\star} \boldsymbol{r}_\pi
$$

です．ここで，$\boldsymbol{r}_\pi \in \mathbb{R}^{|\mathcal{S}|}$ は $r_\pi(s)$をstackしたものです．

定常分布は$s_0$に依存しないので，$\boldsymbol{P}_\pi^{\star}$の行は全て同じであることに気をつけましょう．つまり，

* $v_g(\pi)=\boldsymbol{p}_\pi^{\star} \cdot \boldsymbol{r}_\pi=v_g\left(\pi, s_0\right), \forall s_0 \in \mathcal{S}$
* $\boldsymbol{v}_g(\pi)=v_g(\pi) \cdot \mathbf{1}$

です．

方策$\pi^* \in \Pi_{\mathrm{S}}$は次を満たす時ゲイン最適であると呼びます：

$$
v_g\left(\pi^*, s_0\right) \geq v_g(\pi, s_0), \quad \forall \pi \in \Pi_{\mathrm{S}}, \forall s_0 \in \mathcal{S}, \quad \text { hence } \pi^* \in \underset{\pi \in \Pi_{\mathrm{S}}}{\operatorname{argmax}}\; v_g(\pi)
$$


## ベルマン方程式

平均報酬についても以下のベルマン方程式が成立します：

$$
v_b\left(\pi^*, s\right)+v_g\left(\pi^*\right)=\max _{a \in \mathcal{A}}\left\{r(s, a)+\sum_{c^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, a\right) v_b\left(\pi^*, s^{\prime}\right)\right\}, \quad \forall s \in \mathcal{S},
$$

こおｋで，$v_b$はバイアスと呼ばれ，次で定義されます：

$$
\begin{aligned}
v_b\left(\pi, s_0\right) & :=\lim _{t_{\max } \rightarrow \infty} \mathbb{E}_{S_t, A_t}\left[\sum_{t=0}^{t_{\max }-1}\left(r\left(S_t, A_t\right)-v_g(\pi)\right) \mid S_0=s_0, \pi\right] \\
& =\lim _{t_{\max } \rightarrow \infty} \sum_{t=0}^{t_{\max }-1} \sum_{s \in \mathcal{S}}\left(p_\pi^t\left(s \mid s_0\right)-p_\pi^{\star}(s)\right) r_\pi(s) \\
& =\underbrace{\sum_{t=0}^{\tau-1} \sum_{s \in \mathcal{S}} p_\pi^t\left(s \mid s_0\right) r_\pi(s)}_{\text {the expected total reward } v_\tau}-\tau v_g(\pi)+\underbrace{\lim _{t_{\max } \rightarrow \infty} \sum_{t=\tau}^{t_{\max }-1} \sum_{s \in \mathcal{S}}\left(p_\pi^t\left(s \mid s_0\right)-p_\pi^{\star}(s)\right) r_\pi(s)}_{\text {approaches } 0 \text { as } \tau \rightarrow \infty} \\
& =\sum_{s \in \mathcal{S}}\left\{\lim _{t_{\max } \rightarrow \infty} \sum_{t=0}^{t_{\max }-1}\left(p_\pi^t\left(s \mid s_0\right)-p_\pi^{\star}(s)\right)\right\} r_\pi(s)=\sum_{s \in \mathcal{S}} d_\pi\left(s \mid s_0\right) r_\pi(s),
\end{aligned}
$$

ここで，$d_\pi(s|s_0)$は行列$\boldsymbol{D}_\pi:=\left(\boldsymbol{I}-\boldsymbol{P}_\pi+\boldsymbol{P}_\pi^{\star}\right)^{-1}\left(\boldsymbol{I}-\boldsymbol{P}_\pi^{\star}\right)$と同値です（limitと期待値を入れ替えて良い場合）．

バイアスは以下の解釈ができます：

1. （１行目）$s_0$から始まって$\pi$に従うときの，即時報酬$r(s_t, a_t)$と定常報酬$v_g(\pi)$の差
2. （２行目）$s_0$から始まった過程と，$p_\pi^\star$からサンプルされた初期状態から始まった過程の，報酬の差．よって，$\pi$によって得られる総報酬と，ずっと$v_g(\pi)$の報酬の場合の総報酬の差．
3. （３行目）$v_\tau\left(\pi, s_0\right) \approx v_g(\pi) \tau+v_b\left(\pi, s_0\right)$とみなせるので，期待総報酬$v_\tau$がどれくらい振動するかを表している．
4. （４行目）$\left(p_\pi^t\left(s \mid s_0\right)-p_\pi^{\star}(s)\right)$は定常状態になるまで非ゼロなので，バイアスは一時的な性能とみなせる．

何らかの参照状態$s_{\mathrm{ref}} \in \mathcal{S}$について，相対価値$v_{\text {brel }}$が定義されます：

$$
v_{b r e l}(\pi, s):=v_b(\pi, s)-v_b\left(\pi, s_{\text {ref }}\right)=\lim _{t_{\max } \rightarrow \infty}\left\{v_{t_{\max }}(\pi, s)-v_{t_{\max }}\left(\pi, s_{\text {ref }}\right)\right\}, \quad \forall \pi \in \Pi_{\mathrm{S}}, \forall s \in \mathcal{S} .
$$


## 価値反復法

ベルマン方程式より，$\boldsymbol{v}_b^* \in \mathbb{R}^{\mid \mathcal{S}\mid}$がわかれば最適方策も求まります（このとき，最適ゲイン$v_g^*$も必要になります）．
そのため，目標は$\boldsymbol{v}_b^* \in \mathbb{R}^{\mid \mathcal{S}\mid}$を求めればよいです．
通常の動的計画法と同様にして，$\boldsymbol{v}_b^*$の代わりに$\boldsymbol{q}_b^*\in \mathbb{R}^{|\mathcal{S}||\mathcal{A}|}$を求めます．
